In [1]:
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sys
import itertools
from collections import namedtuple
from pathlib import Path

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 1
%aimport analyze

with open('plots/style.json') as f:
    mpl.rcParams.update(json.load(f))

## Wildstyle failures

In [2]:
ws = analyze.Analysis('results/wildstyle')
print(ws)

Analysis for Celeritas v0.2.0 on wildstyle


In [3]:
ftab = analyze.make_failure_table(ws.failures())
ftab.to_frame()

,Failure
cms2018+field+msc/vecgeom+cpu (0),internal assertion failed: `mfp > 0` at `Along...
cms2018+field+msc/vecgeom+cpu (1),internal assertion failed: `mfp > 0` at `Along...
cms2018+field+msc/vecgeom+gpu (0),`celeritas: internal assertion failed: succeeded`
cms2018+field+msc/vecgeom+gpu (1),`celeritas: internal assertion failed: succeeded`
cms2018/vecgeom+cpu (0),internal assertion failed: `speed > 0` at `Alo...
cms2018/vecgeom+cpu (1),internal assertion failed: `speed > 0` at `Alo...
cms2018/vecgeom+gpu (0),`celeritas: internal assertion failed: speed > 0`
cms2018/vecgeom+gpu (1),`celeritas: internal assertion failed: speed > 0`
simple-cms+field+msc/orange+gpu (1),`celeritas: internal assertion failed: is_soft...
simple-cms+field+msc/vecgeom+cpu (0),internal assertion failed: `step_limit->action...


In [4]:
summed = analyze.summarize_instances(ws.result[['unconverged']])

Unconverged tracks (from runs that didn't fail):

In [5]:
unconv = summed[('unconverged', 'mean')]
unconv[unconv > 0].unstack('arch') 

,arch,cpu,gpu
problem,geo,,
simple-cms+field+msc,orange,0.5,2.0


## Summit results

In [6]:
summit = analyze.Analysis('results/summit')
print(summit)
summed = analyze.summarize_instances(summit.result[summit.successful].dropna(how='all'))

Analysis for Celeritas v0.2.0 on summit


In [7]:
deets = summit.load_results(('testem3-flat','orange','gpu'), 0)
deets['system']['device']

{'capability_major': 7,
 'capability_minor': 0,
 'clock_rate': 1530000,
 'default_block_size': 256,
 'device_id': 0,
 'eu_per_cu': 1,
 'max_blocks_per_grid': 2147483647,
 'max_blocks_per_multiprocessor': 32,
 'max_cache_size': 6291456,
 'max_threads_per_block': 1024,
 'max_threads_per_cu': 2048,
 'memory_clock_rate': 877000,
 'multiprocessor_count': 80,
 'name': 'Tesla V100-SXM2-16GB',
 'platform': 'cuda',
 'regs_per_block': 65536,
 'regs_per_multiprocessor': 65536,
 'shared_mem_per_block': 49152,
 'threads_per_warp': 32,
 'total_const_mem': 65536,
 'total_global_mem': 16911433728}

### Failures

Average number of unconverged tracks:

In [8]:
unconv = analyze.summarize_instances(summit.result['unconverged'])['mean']
unconv[unconv > 0].unstack('arch') 

arch                          cpu       gpu
problem              geo                   
cms2018+field+msc    vecgeom  1.5  9.666667
simple-cms+field     orange   NaN  0.666667
simple-cms+field+msc orange   1.0  0.333333
                     vecgeom  0.5  0.166667

### Timing tables

In [9]:
(fig, [run_ax, setup_ax]) = plt.subplots(nrows=2,
                                         gridspec_kw=dict(height_ratios=[3, 1]),
                                         subplot_kw=dict(yscale='log'))
summit.plot_results(run_ax, summed['total_time'])
run_ax.legend();
run_ax.set_ylabel('Run [s]')
run_ax.tick_params(labelbottom=False)
summit.plot_results(setup_ax, summed['setup_time'])
setup_ax.set_ylabel('Setup [s]')
analyze.annotate_metadata(run_ax, summit)
fig.savefig('plots/timing.pdf', transparent=True)
plt.close()

In [10]:
def float_fmt_transform(digits):
    format = "{{:.{}f}}".format(digits).format
    def transform(val):
        if np.isnan(val):
            return "---"
        return format(val)
    return transform

times = summed[('total_time', 'mean')].unstack()
times.style.format(float_fmt_transform(2))

In [11]:
event_rate = analyze.calc_event_rate(summit, summed)
event_rate['mean'].xs('simple-cms+field+msc', level='problem').unstack('arch')

arch,cpu,gpu
geo,,
orange,0.077120,0.823527
vecgeom,0.078286,0.810254


In [12]:
speedup = analyze.get_cpugpu_ratio(summed['total_time'])
speedup.dropna().style.format(float_fmt_transform(1))

### Plots

In [13]:
problems = summit.problems()
problem_to_abbr = summit.problem_to_abbr(problems)
p_to_i = dict(zip(problems, itertools.count()))

In [14]:
fig, ax = plt.subplots()
summit.plot_results(ax, speedup)
ax.set_ylabel("Speedup (7-CPU / 1-GPU wall time)")
ax.set_ylim([0, None])
analyze.annotate_metadata(ax, summit);
fig.savefig('plots/speedups.pdf', transparent=True)
plt.close()

In [15]:
fig, axes = plt.subplots(nrows=2, figsize=(4,4), subplot_kw=dict(yscale='log'))
for (ax, q) in zip(axes, ['step', 'primary']):
    summit.plot_results(ax, analyze.inverse_summary(summed['avg_time_per_' + q]))
    ax.set_ylabel(q + ' per sec')
    ax.legend()
fig.savefig('plots/steps-vs-primaries.png', dpi=300)
plt.close()

In [16]:
(fig, (time_ax, occ_ax)) = plt.subplots(
    nrows=2, figsize=(4, 4),
    gridspec_kw=dict(height_ratios=[3, 1])
)
time_ax.set_yscale('log')
summit.plot_results(time_ax, event_rate)
time_ax.set_ylabel(r"Event rate [1/s]")
time_ax.set_ylim([0.5 * event_rate['mean'].min(), None])
time_ax.legend()
time_ax.tick_params(labelbottom=False)
summit.plot_results(occ_ax, summed['slot_occupancy'])
occ_ax.set_ylabel("Slot occupancy")
analyze.annotate_metadata(ax, summit)
fig.savefig('plots/rate-occupancy.pdf', transparent=True)
plt.close()

## Action fraction pie charts


In [17]:
avg_time = summed[('total_time', 'mean')].T
mean_action_times = summit.action_times().xs('mean', axis=1, level=1).T

In [18]:
mean_action_times.divide(avg_time, axis=1).dropna(how='all', axis=1).style.format(float_fmt_transform(2))

In [19]:
mean_action_times.columns.get_level_values('problem')

Index(['cms2018', 'cms2018', 'cms2018+field+msc', 'simple-cms+field',
       'simple-cms+field', 'simple-cms+field+msc', 'simple-cms+field+msc',
       'simple-cms+field+msc', 'simple-cms+field+msc', 'simple-cms+msc',
       'simple-cms+msc', 'testem15', 'testem15', 'testem15+field',
       'testem15+field', 'testem15+field+msc', 'testem15+field+msc',
       'testem15+field+msc', 'testem15+field+msc', 'testem3-flat',
       'testem3-flat', 'testem3-flat', 'testem3-flat', 'testem3-flat+field',
       'testem3-flat+field', 'testem3-flat+msc'],
      dtype='object', name='problem')

In [20]:
for prob in ["cms2018", "cms2018+field+msc"]:
    geo = 'vecgeom'
    if prob not in mean_action_times.columns.get_level_values('problem'):
        print("Missing problem:", prob)
        continue
    temp = mean_action_times.xs((prob, geo), axis=1, level=('problem', 'geo')).dropna()
    for (arch, series) in temp.items():
        (fig, ax) = plt.subplots(figsize=(4, 4))
        ax.pie(series, labels=series.index, autopct='%1.1f%%', pctdistance=0.85)
        ax.axis('equal')
        name = (prob, geo, arch)
        slashname = "/".join(name)
        fig.text(
            0.98, 0.02, f"{slashname}\n{summit.version} on {summit.system}",
            va='bottom', ha='right',
            fontstyle='italic', color=(0.5,)*3, size='xx-small',
            zorder=-100
        )
        dashname = "-".join(name)
        fig.savefig(f'plots/actions-{dashname}.pdf', transparent=True)
        plt.close()

### Plot per-step timing on GPU

In [21]:
cms = [summit.load_results((p, 'vecgeom', 'gpu'), 0)
       for p in ['cms2018', 'cms2018+field+msc']]

for plot, label in [(analyze.plot_counts, 'counts'),
                    (analyze.plot_accum_time, 'time')]:
    (fig, axes) = plt.subplots(ncols=2, figsize=(8, 2))
    
    for (i, ax, data) in zip(itertools.count(), axes, cms):
        objs = plot(ax, data)
        analyze.annotate_metadata(ax, data['_metadata'])
        if i == 0:
            objs['oax'].set_ylabel(None)
        elif i == 1:
            objs['ax'].set_ylabel(None)
    fig.savefig(f'plots/cms-{label}.pdf', transparent=True)
    plt.close()

## Crusher

In [22]:
crusher = analyze.Analysis('results/crusher')
print(crusher)

Analysis for Celeritas v0.2.0-1+49ccc7c8 on crusher


In [23]:
# VecGeom failures aren't really failures; just missing capability
failures = crusher.failures().xs('orange', level='geo').fillna(1)
failures.groupby(['problem', 'arch']).count().unstack()

failure              stderr      stdout     
arch                    cpu  gpu    cpu  gpu
problem                                     
simple-cms+field+msc    2.0  NaN    2.0  NaN
testem3-flat+field      4.0  4.0    4.0  4.0
testem3-flat+msc        7.0  8.0    7.0  8.0

In [24]:
csum = analyze.summarize_instances(crusher.result[crusher.successful].dropna(how='all'))

In [25]:
csum[('total_time', 'mean')].unstack()

,arch,cpu,gpu
problem,geo,,
simple-cms+field,orange,86.364525,7.079069
simple-cms+field+msc,orange,88.769386,10.412078
simple-cms+msc,orange,83.134969,2.175320
testem15,orange,89.462248,1.660482
testem15+field,orange,59.999452,1.850357
testem15+field+msc,orange,89.137889,2.165253
testem3-flat,orange,107.263641,3.244948
testem3-flat+field,orange,140.217632,6.640344
testem3-flat+msc,orange,174.890197,NaN


In [26]:
rel_err = csum.xs('std', axis=1, level=1) / csum.xs('mean', axis=1, level=1)
high_err = rel_err > 0.02
rel_err[high_err].dropna(how='all').dropna(how='all', axis=1)

avg_time_per_primary  avg_time_per_step  \
problem              geo    arch                                            
simple-cms+field     orange cpu               0.189675           0.189687   
                            gpu               0.109828           0.109866   
simple-cms+field+msc orange cpu               0.242730           0.242599   
                            gpu               0.382798           0.382813   
simple-cms+msc       orange cpu               0.172376           0.172437   
                            gpu                    NaN                NaN   
testem15             orange cpu               0.422068           0.422085   
                            gpu                    NaN                NaN   
testem15+field       orange cpu               0.321016           0.321043   
                            gpu                    NaN                NaN   
testem15+field+msc   orange cpu               0.164774           0.164771   
                            gpu                    NaN                NaN   
testem3-flat         orange cpu               0.539319           0.539219   
                            gpu                    NaN                NaN   
testem3-flat+field   orange cpu               0.348533           0.348523   
                            gpu                    NaN                NaN   

                                  num_step_iters  pre_emptying_time  \
problem              geo    arch                                      
simple-cms+field     orange cpu              NaN           0.478345   
                            gpu         0.039383           0.294876   
simple-cms+field+msc orange cpu              NaN           0.533760   
                            gpu         0.297543           0.033697   
simple-cms+msc       orange cpu              NaN           0.431574   
                            gpu         0.034899                NaN   
testem15             orange cpu              NaN           0.426240   
                            gpu         0.026858                NaN   
testem15+field       orange cpu              NaN           0.454078   
                            gpu         0.026858                NaN   
testem15+field+msc   orange cpu              NaN           0.308426   
                            gpu         0.024753                NaN   
testem3-flat         orange cpu              NaN           0.543723   
                            gpu              NaN                NaN   
testem3-flat+field   orange cpu              NaN           0.420604   
                            gpu         0.031953           0.043266   

                                  setup_time  slot_occupancy  total_time  
problem              geo    arch                                          
simple-cms+field     orange cpu     0.856062             NaN    0.189675  
                            gpu     0.030434        0.037875    0.109828  
simple-cms+field+msc orange cpu     0.049217             NaN    0.242730  
                            gpu          NaN        0.232652    0.382798  
simple-cms+msc       orange cpu     0.884673             NaN    0.172376  
                            gpu     0.036296        0.035145         NaN  
testem15             orange cpu     0.775743             NaN    0.422068  
                            gpu     0.043046        0.026928         NaN  
testem15+field       orange cpu          NaN             NaN    0.321016  
                            gpu     0.032466        0.026928         NaN  
testem15+field+msc   orange cpu          NaN             NaN    0.164774  
                            gpu     0.034051        0.024806         NaN  
testem3-flat         orange cpu     0.916055             NaN    0.539319  
                            gpu     0.033349             NaN         NaN  
testem3-flat+field   orange cpu          NaN             NaN    0.348533  
                            gpu     0.044499        0.031954         NaN

In [27]:
analyze.get_cpugpu_ratio(csum['total_time'])

,,mean,std
problem,geo,,
simple-cms+field,orange,12.199984,2.673957
simple-cms+field+msc,orange,8.525616,3.864392
simple-cms+msc,orange,38.217353,6.601693
testem15,orange,53.877265,22.741743
testem15+field,orange,32.425875,10.413711
testem15+field+msc,orange,41.167420,6.786948
testem3-flat,orange,33.055577,17.828039
testem3-flat+field,orange,21.116020,7.360462
testem3-flat+msc,orange,NaN,NaN


In [28]:
crusher_times = csum['total_time']
crusher_times

count        mean        std
problem              geo    arch                              
simple-cms+field     orange cpu     8.0   86.364525  16.381155
                            gpu     7.0    7.079069   0.777478
simple-cms+field+msc orange cpu     6.0   88.769386  21.546988
                            gpu     5.0   10.412078   3.985723
simple-cms+msc       orange cpu     8.0   83.134969  14.330453
                            gpu     8.0    2.175320   0.024413
testem15             orange cpu     8.0   89.462248  37.759161
                            gpu     8.0    1.660482   0.008983
testem15+field       orange cpu     8.0   59.999452  19.260794
                            gpu     8.0    1.850357   0.017432
testem15+field+msc   orange cpu     8.0   89.137889  14.687576
                            gpu     8.0    2.165253   0.011692
testem3-flat         orange cpu     8.0  107.263641  57.849365
                            gpu     8.0    3.244948   0.013421
testem3-flat+field   orange cpu     4.0  140.217632  48.870489
                            gpu     4.0    6.640344   0.034761
testem3-flat+msc     orange cpu     1.0  174.890197        NaN

In [29]:
crusher_rates = analyze.calc_event_rate(crusher, csum)
summit_rates = analyze.calc_event_rate(summit, summed.loc[crusher_times.index])

counts = {
    ('summit', 'cpu'): 7,
    ('summit', 'gpu'): 1,
    ('crusher', 'cpu'): 8,
    ('crusher', 'gpu'): 1,
}

In [30]:
(crusher_rates['mean'] / summit_rates['mean']).unstack()

,arch,cpu,gpu
problem,geo,,
simple-cms+field,orange,0.839008,0.821999
simple-cms+field+msc,orange,1.022506,0.816362
simple-cms+msc,orange,0.860179,1.444362
testem15,orange,0.534269,1.471178
testem15+field,orange,1.023794,1.291316
testem15+field+msc,orange,0.858386,1.278250
testem3-flat,orange,0.819027,1.044006
testem3-flat+field,orange,1.073450,0.859865
testem3-flat+msc,orange,0.921367,NaN


In [31]:
fig, ax = plt.subplots()
ax.set_yscale('log')
for offset, color, machine, rates in [(-0.05, '#7A954F', 'Summit', summit_rates),
                                      (0.05, '#BC5544', 'Crusher', crusher_rates)]:
    for arch in ['cpu', 'gpu']:
        summary = rates.xs(arch, level='arch')
        index = np.array([p_to_i[p]
                          for p in summary.index.get_level_values('problem')], dtype=float)
        index += offset
    
        mark = analyze.ARCH_SHAPES[arch]
        count = counts[(machine.lower(), arch)]
        arch = arch.upper()
        ax.errorbar(index, summary['mean'], summary['std'],
                    capsize=0, fmt='none', ecolor=(0.2,)*3)
        scat = ax.scatter(index, summary['mean'], c=color, marker=mark,
                         label=f"{machine} ({count} {arch})")    
xax = ax.get_xaxis()
xax.set_ticks(np.arange(len(problems)))
xax.set_ticklabels(list(problem_to_abbr.values()), rotation=90)
grid = ax.grid()
ax.set_axisbelow(True)
ax.legend()
ax.set_ylabel(r"Event rate [1/s]")
analyze.annotate_metadata(ax, summit)
fig.savefig('plots/crusher-vs-summit.pdf')
plt.close()